In [ ]:
data_date = "2018_06_27_13_49_07"
data_dir = "/home/pleroy/DATA/2018_06_27_LETG/2018_06_27/jde/" + data_date
# selection in record
firstFile = 0
nbFiles = 379

# Initialization

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt5

import sys
import gc

sys.path.insert(0, "/home/pleroy/DEV/processing/PoSAR-MC")
from posarmctools.readdata import *

%load_ext autoreload
%autoreload 2

sys.path.insert(0, "/home/pleroy/DEV/processing/focalization_python")
from posarutils.other.read_data_and_build_rd import *
from posarutils.other.PosarMCParameters import *

# Parameters

In [ ]:
params_filename = data_dir + "/" + data_date + "_parameters.xml"
params = PosarMCParameters_v2( params_filename )
params.print()

In [ ]:
buffersPerFile = params.buffersPerFile
samplesPerRamp = params.samplesPerRamp
rampsPerFile = params.rampsPerFile
samplesPerFile = samplesPerRamp * rampsPerFile

print( "buffersPerFile = {}, samplesPerRamp = {}, rampsPerFile = {}".format(
    buffersPerFile, samplesPerRamp, rampsPerFile )
     )

lastFile = firstFile + nbFiles - 1
firstBuffer = firstFile * buffersPerFile

In [ ]:
lastFile

# Read the data

In [ ]:
scalingFactor = 2 / 65535
offset = -32768

In [ ]:
timeSerie_A = np.zeros( samplesPerFile )
adc_A = np.zeros( (nbFiles, samplesPerFile) )

In [ ]:
lastBuffer = (nbFiles-1) * buffersPerFile + firstBuffer
for k in range( nbFiles ):
    
    # read the data
    nb = str( int( k * buffersPerFile + firstBuffer) )
    filename = data_dir + "/record" + nb + ".bin"
    readFileADLINKCh0( filename, samplesPerFile, timeSerie_A )
    
    print("k = {}, block = {} / {}".format(k, nb, lastBuffer))
    
    adc_A[ k, : ] = (timeSerie_A + offset) * scalingFactor

In [ ]:
A_reshaped = adc_A.reshape(nbFiles * rampsPerFile, samplesPerRamp)

**Check that the first ramp is an up ramp**

In [ ]:
plt.figure()
plt.plot( A_reshaped[0, :] )
title = "first acquisition " + data_date
plt.title(title)
plt.grid()
plt.savefig( data_dir + "/" + title + ".png", bbox_inches='tight')

**Set the shifted flag properly**

In [ ]:
rampDownFirst = 1

# Build RD

In [ ]:
#RDc = build_rd_from_data( params, A_reshaped, shifted )
withHanning = 1
RDc = build_rd_from_data_rampDown( params, A_reshaped, rampDownFirst, withHanning )

In [ ]:
A_reshaped = None
adc_A = None
gc.collect()

In [ ]:
coupling_RD = np.average(RDc, 0)
if withHanning:
    np.save( data_dir + "/coupling_RD_files_{}_{}_rampDown_hanning".format(firstFile, lastFile), coupling_RD )
else:
    np.save( data_dir + "/coupling_RD_files_{}_{}_rampDown".format(firstFile, lastFile), coupling_RD )

In [ ]:
plt.figure()
plt.plot(np.abs(coupling_RD), 'k', label='coupling')
plt.legend()
plt.grid()

In [ ]:
# save RD data
RD = np.fft.ifftshift(RDc, 1)

In [ ]:
np.save(data_dir + '/np_fft_ifftshift(RD1, 1)_{}_{}'.format(firstFile, lastFile), RD) 